# Building inverted index and answering queries

In the first part of the lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

## Preprocessing

First, we need a unified approach to documents and queries preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library, `spacy`, or any other you know.

In [1]:
import nltk

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        
        return nltk.word_tokenize(text)
        return ['one', 'two', 'three']

    
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        return stemmer.stem(word)
        return 'stemmed_word'

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        return word not in self.stop_words and word.isalpha()
        return False

    
    def preprocess(self, text):
        #TODO combine all previous methods together: tokenize lowercased text 
        # and stem it, ignoring not appropriate words
        tokenized = self.tokenize(text.lower())
        return [self.stem(w, self.ps) for w in tokenized if self.is_apt_word(w)]
        
        return ['one', 'two', 'three']

### Tests

In [2]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## Crawling and Indexing

### Base classes

Here are some base classes you will need for writing an indexer. The code is given, still, you need to change some. Namely, the `parse` method (it is also possible to use your own implementations of other methods, just make sure they work). The reason to change is that the method always makes complete parsing, which we want to avoid, when we only need e.g. links or a specific portions of text.

In [3]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        # this code is not supposed to be good :) 
        # Please discuss why this line is bad
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlReutersArticle(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self):
        #TODO change this method
        
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
   
        model = BeautifulSoup(self.content, "html.parser")
        
        self.anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            text = anchor.text
            self.anchors.append((text, href))
            
        # extract only header and main content
        # discuss why using classes like article-body__content__17Yit is
        # wrong strategy
        header = model.find('h1')
        content = model.find('p')
        if content:
            content = content.parent
                        
        if header is None or content is None:
            self.article_text = ""
            return
        
        texts = header.findAll(text=True) + content.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        self.article_text = "\n".join(t.strip() for t in visible_texts)

In [4]:
doc = HtmlReutersArticle("https://www.reuters.com/business/healthcare-pharmaceuticals/amazon-launches-virtual-healthcare-clinic-us-2022-11-15/")
doc.download()
doc.parse()
doc.article_text

"Amazon launches virtual healthcare clinic in U.S. for common ailments\nNov 15 (Reuters) - Amazon.com Inc\n(AMZN.O)\non Tuesday launched Amazon clinic, a virtual platform where users can connect with healthcare providers to help treat common ailments like allergies and skin conditions.\nAmazon has for years sought to expand its presence in healthcare. It bought online pharmacy PillPack in 2018, underpinning a prescription delivery and price-comparison site it later launched as Amazon Pharmacy, which lets users buy over-the-counter drugs via Prime memberships.\nAmazon\nsaid\nits new service would operate in 32 states, where customers who seek treatment, will be connected to healthcare providers. The service does not include health insurance and pricing will vary depending on providers, it added.\nThe online retailer first piloted virtual care visits for its own staff in Seattle in 2019 before offering services to other employers under the Amazon Care brand, which it now plans to close d

### Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (starting with https://www.reuters.com/...) for visiting. To speed up, do not consider pages with content type other than `html`: `.pdf`, `.mp3`, `.avi`, `.mp4`, `.txt`, ... . If crawler encounters an article page (of a form https://www.reuters.com/world/...), it saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`: `{doc_id : url}`
    - `index`: `{term : [collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]}`
    - `doc_lengths`: `{doc_id : doc_length}` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.

Your crawler have to print visited urls as it runs.

In [5]:
from collections import Counter
from queue import Queue
import pickle
import os
import re

class ReutersSpecificIndexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
        
    
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        
        q = Queue()
        q.put((source, 0))
        visited = set()
        doc_counter = 0
        # creating a folder if needed
        if not os.path.exists(collection_path):
            os.makedirs(collection_path)
        # doing a BFS
        while not q.empty():
            url, url_depth = q.get()
            if url not in visited:
                visited.add(url)
                try:
                    doc = HtmlReutersArticle(url)    # download and parse url
                    if doc.download():
                        doc.parse()

                        if re.match(r'https\://www\.reuters\.com/\w+/\w+', url):
                        # if url.startswith("https://www.reuters.com/"):
                            doc.persist(collection_path)
                            self.doc_urls[doc_counter] = url
                            self.index_doc(doc, doc_counter)
                            doc_counter += 1                        
                            yield doc
                            if limit is not None and doc_counter == limit:
                                return
                            
                            # filter links, consider only inner html pages
                        if url_depth + 1 < depth:
                            valid_anchors = filter(self.accepted_url, doc.anchors)
                            for a in valid_anchors:
                                q.put((a[1], url_depth + 1))
    
                except FileNotFoundError as e:
                    print("Analyzing", url, "led to FileNotFoundError")
                    
    
    def accepted_url(self, anchor):
        url = str(anchor[1])
        if not url.startswith("https://www.reuters.com/"):
            return False
        if url[-4:]  in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
            return False
        return True
        
        
    def index_doc(self, doc, doc_id):
        # add documents to index
        doc.parse()
        # preprocess - tokenize, remove stopwords and non-alphanumeric tokens and stem
        content = self.prep.preprocess(doc.article_text)
        self.doc_lengths[doc_id] = len(content)
        # get dict of terms in current article
        article_index = Counter(content)
        # update global index
        for term in article_index.keys():
            article_freq = article_index[term]
            if term not in self.index:                
                self.index[term] = [article_freq, (doc_id, article_freq)]
            else:
                self.index[term][0] += article_freq
                self.index[term].append((doc_id, article_freq))

### Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [6]:
indexer = ReutersSpecificIndexer()
k = 1
for c in indexer.crawl_generator_for_index(
        source="https://www.reuters.com/technology", 
        depth=2, 
        collection_path="test_collection", 
        limit=15):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/technology/ftx-officials-contact-with-us-regulators-filing-2022-11-15/
2 https://www.reuters.com/business/healthcare-pharmaceuticals/amazon-launches-virtual-healthcare-clinic-us-2022-11-15/
3 https://www.reuters.com/business/healthcare-pharmaceuticals/
4 https://www.reuters.com/markets/deals/buffetts-berkshire-discloses-big-taiwan-semi-stake-2022-11-14/
5 https://www.reuters.com/technology/chinas-tencent-starts-new-round-layoffs-sources-2022-11-15/
6 https://www.reuters.com/technology/cryptoverse-so-long-solana-ether-rival-clobbered-by-ftx-crash-2022-11-15/
7 https://www.reuters.com/business/future-of-money/
8 https://www.reuters.com/business/finance/exclusive-activist-impactive-eyes-proxy-fight-envestnet-amid-sluggish-stock-2022-11-15/
9 https://www.reuters.com/technology/youtube-expands-shopping-features-following-digital-ad-slowdown-ft-2022-11-15/
10 https://www.reuters.com/technology/japan-unicorn-opn-buys-merchante-enter-us-online-payment-sector-2022-11-1

Please test these documents contain a desired stem (or its derivate):

In [7]:
some_stem = 'bank'
print(indexer.index[some_stem])
for pair in indexer.index[some_stem][1:]:
    print(indexer.doc_urls[pair[0]])

[6, (0, 2), (3, 1), (5, 1), (7, 1), (9, 1)]
https://www.reuters.com/technology/ftx-officials-contact-with-us-regulators-filing-2022-11-15/
https://www.reuters.com/markets/deals/buffetts-berkshire-discloses-big-taiwan-semi-stake-2022-11-14/
https://www.reuters.com/technology/cryptoverse-so-long-solana-ether-rival-clobbered-by-ftx-crash-2022-11-15/
https://www.reuters.com/business/finance/exclusive-activist-impactive-eyes-proxy-fight-envestnet-amid-sluggish-stock-2022-11-15/
https://www.reuters.com/technology/japan-unicorn-opn-buys-merchante-enter-us-online-payment-sector-2022-11-15/


### 1.2.4. Building an index

In [8]:
indexer = ReutersSpecificIndexer()
for k, c in enumerate(
                indexer
                    .crawl_generator_for_index(
                        "https://www.reuters.com/", 
                        3, 
                        "docs_collection",
                        limit=100   # optional limit
                    )):
    print(k + 1, c.url)

1 https://www.reuters.com/markets/global-market-data
2 https://www.reuters.com/markets/us/home-depot-beats-sales-estimates-steady-demand-2022-11-15/
3 https://www.reuters.com/markets/macromatters/
4 https://www.reuters.com/world/china/chinas-factory-output-retail-sales-miss-forecasts-economy-losing-steam-2022-11-15/
5 https://www.reuters.com/business/future-of-money/
6 https://www.reuters.com/technology/ftxs-new-ceo-helped-bolster-enron-victims-recovery-2022-11-15/
7 https://www.reuters.com/business/retail-consumer/walmart-expects-smaller-drop-annual-profit-announces-20-bln-share-buyback-2022-11-15/
8 https://www.reuters.com/legal/elon-musk-trial-opens-decide-fate-his-56-bln-tesla-pay-2022-11-14/
9 https://www.reuters.com/markets/deals/buffetts-berkshire-discloses-big-taiwan-semi-stake-2022-11-14/
10 https://www.reuters.com/technology/cryptoverse-so-long-solana-ether-rival-clobbered-by-ftx-crash-2022-11-15/
11 https://www.reuters.com/world/g20-starts-bali-ukraine-war-raging-inflation-t

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


50 https://www.reuters.com/tools/mobile/us
51 https://www.reuters.com/world/europe/ukraine-hails-chinas-opposition-nuclear-threats-2022-11-15/
52 https://www.reuters.com/world/europe/
53 https://www.reuters.com/technology/exclusive-russian-software-disguised-american-finds-its-way-into-us-army-cdc-2022-11-14/
54 https://www.reuters.com/world/middle-east/uae-official-calls-unambivalent-us-security-commitment-2022-11-14/
55 https://www.reuters.com/world/middle-east/
56 https://www.reuters.com/world/europe/both-russia-ukraine-tortured-prisoners-war-un-says-2022-11-15/
57 https://www.reuters.com/world/china/china-reports-17909-new-covid-cases-nov-14-vs-16203-day-earlier-2022-11-15/
58 https://www.reuters.com/world/china/
59 https://www.reuters.com/world/europe/exclusive-germany-steps-up-emergency-cash-plans-cope-blackout-sources-2022-11-15/
60 https://www.reuters.com/world/asia-pacific/xi-biden-knock-off-democracy-vs-autocracy-talk-2022-11-14/
61 https://www.reuters.com/world/europe/ukrain

### Index statistics

Load the index and print the statistics.

In [9]:
print('Total index keys count', len(indexer.index))

print('\nTop stems by number of documents they apperared in:')
sorted_by_n_docs = sorted(indexer.index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])

print('\nTop stems by overall frequency:')
sorted_by_freq = sorted(indexer.index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index keys count 4113

Top stems by number of documents they apperared in:
[('reuter', 61), ('trust', 56), ('thomson', 56), ('standard', 56), ('said', 56), ('principl', 56), ('our', 56), ('nov', 53), ('report', 52), ('edit', 48), ('have', 47), ('which', 43), ('not', 43), ('year', 42), ('had', 42), ('monday', 40), ('after', 40), ('more', 39), ('also', 39), ('their', 38)]

Top stems by overall frequency:
[('said', 336), ('reuter', 174), ('have', 131), ('not', 130), ('had', 115), ('which', 103), ('report', 100), ('more', 96), ('compani', 93), ('year', 88), ('firm', 87), ('ukrain', 84), ('after', 84), ('their', 81), ('china', 81), ('twitter', 72), ('than', 72), ('new', 72), ('law', 72), ('would', 71)]


## Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [10]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        #TODO retrieve a set of documents containing all query terms
        
        # retrieve a set of documents containing all query terms
        postings = []
        for term in query.keys():
            if term not in index:  # ignoring absent terms
                continue
            posting = index[term][1:]
            # extract document info only
            posting = [i[0] for i in posting]
            postings.append(posting)
        docs = set.intersection(*map(set,postings))
        return docs 
        
        return {0, 1, 3}

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores
        
        scores = {}
        N = len(doc_lengths)
        avgdl = sum(doc_lengths.values()) / float(len(doc_lengths))
        for term in query.keys():
            if term not in index:  # ignoring absent terms
                continue
            n_docs = len(index[term]) - 1
            idf = math.log10((N - n_docs + 0.5) / (n_docs + 0.5))
            postings = index[term][1:]
            for posting in postings:
                doc_id = posting[0]
                doc_tf = posting[1]
                score = idf * doc_tf * (k1 + 1) / (doc_tf + k1 * (1 - b + b * (doc_lengths[doc_id] / avgdl)))
                if doc_id not in scores:
                    scores[doc_id] = score
                else:  # accumulate scores
                    scores[doc_id] += score
        return scores
        
        return {0: 0.32, 5: 1.17}

### Tests

In [11]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

### And now query the real index

In [16]:
import time

queries = ["russia ukraine", "Hong Kong", "crypto money"]
for q in queries:
    print(q)
    qobj = QueryProcessing.prepare_query(q)
    for res in QueryProcessing.boolean_retrieval(qobj, indexer.index):
        print('\t', indexer.doc_urls[res])

    s = time.time()
    okapi_res = QueryProcessing.okapi_scoring(qobj, indexer.doc_lengths, indexer.index)
    e = time.time()
    print(f"\t == Okapi Time: {e - s:.5f} ==")
    for res in okapi_res:
        print('\t', indexer.doc_urls[res], okapi_res[res])
    
    print()

russia ukraine
	 https://www.reuters.com/business/energy/china-refiners-slow-down-russian-oil-purchases-sanctions-near-trade-2022-11-14/
	 https://www.reuters.com/world/g20-starts-bali-ukraine-war-raging-inflation-top-agenda-2022-11-15/
	 https://www.reuters.com/world/asia-pacific/g20-summit-latest-biden-xi-meet-2022-11-14/
	 https://www.reuters.com/world/ahead-tense-g20-summit-biden-xi-meet-talks-2022-11-14/
	 https://www.reuters.com/world/most-g20-members-strongly-condemn-war-ukraine-draft-declaration-says-2022-11-15/
	 https://www.reuters.com/world/inside-besieged-mariupol-left-ruins-after-russian-bombardment-2022-03-24/
	 https://www.reuters.com/markets/global-markets-view-usa-2022-11-15/
	 https://www.reuters.com/world/europe/why-stop-now-ukraine-seen-pressing-advantage-after-kherson-victory-2022-11-14/
	 https://www.reuters.com/world/europe/ukraine-hails-chinas-opposition-nuclear-threats-2022-11-15/
	 https://www.reuters.com/world/europe/
	 https://www.reuters.com/world/europe/bo